In [ ]:
import random
import torch
import numpy as np
import os
import networkx as nx
import argparse
from utils import Normalizer
from copy import deepcopy
from utils import classification_eval
'''from data_process import load_data_coordinate

from data_process import load_feature_matrix

from data_process import padding_zeros_matrix

from data_process import edge_number_matrix

#from data_process import GFA_label'''

import torch.utils.data as Data

from torch.utils.data import random_split

from layer import GNNLayer

from remodel import GNNModel

import matplotlib.pyplot as plt
plt.switch_backend('agg')

class GaussianDistance(object):
    """
    Expands the distance by Gaussian basis.
    Inspired by Cgcnn, "https://github.com/txie-93/cgcnn"
    """
    def __init__(self, dmin, dmax, step, var=None):
        """
        Args:
            dmin (float): Minimum interatomic distance
            dmax (float): Maximum interatomic distance
            step (float): Step size for the Gaussian filter
        """
        assert dmin < dmax
        assert dmax - dmin > step
        self.filter = np.arange(dmin, dmax + step, step)
        self.var = var if var else step

    def expand(self, distances):
        """
        Apply Gaussian distance filter to a distance array
        Args:
            distances (Array, (N)): A distance array

        Returns:
            expanded_distance (Array, (N, len(self.filter)): Expanded distance
                array using the Gaussian filter.
        """
        return np.exp(-(np.expand_dims(distances, axis=-1) - self.filter)**2 /
                      self.var**2)



#edge_feature_matrix_list,neighbor_node_number_matrix_list,neighbor_node_index_matrix_list,node_feature_matrix_list,contre_node_index_matrix_list = load_feature_matrix(num_samples=100,
                                                                                                                      #atom_number=4000,coordinate_number=3,threshold=4.0)


#np.save("cu5zr5_edge_feature_matrix_list.npy", edge_feature_matrix_list)

#np.save("cu5zr5_node_feature_matrix_list.npy", node_feature_matrix_list)

#np.save("cu5zr5_neighbor_node_number_matrix_list.npy",neighbor_node_number_matrix_list)

#np.save("cu5zr5_neighbor_node_index_matrix_list.npy",neighbor_node_index_matrix_list)

#np.save("cu5zr5_contre_node_index_matrix_list.npy",contre_node_index_matrix_list)


print('###################################################Save_data_success#############################################################')

#edge_feature_matrix_list=np.load("edge_feature_matrix_list.npy")

#node_feature_matrix_list=np.load("node_feature_matrix_list.npy")

#neighbor_node_number_matrix_list=np.load("neighbor_node_number_matrix_list.npy")

#neighbor_node_index_matrix_list=np.load("neighbor_node_index_matrix_list.npy")

#contre_node_index_matrix_list=np.load("contre_node_index_matrix_list.npy",allow_pickle=True)

cu5zr5_edge_feature_matrix_list = np.load("cu5zr5_edge_feature_matrix_list.npy")

cu5zr5_node_feature_matrix_list = np.load("cu5zr5_node_feature_matrix_list.npy")

cu5zr5_neighbor_node_index_matrix_list = np.load("cu5zr5_neighbor_node_index_matrix_list.npy")

cu5zr5_neighbor_node_number_matrix_list = np.load("cu5zr5_neighbor_node_number_matrix_list.npy")


ni5al5_edge_feature_matrix_list = np.load("ni5al5_edge_feature_matrix_list.npy")

ni5al5_node_feature_matrix_list = np.load("ni5al5_node_feature_matrix_list.npy")

ni5al5_neighbor_node_index_matrix_list = np.load("ni5al5_neighbor_node_index_matrix_list.npy")

ni5al5_neighbor_node_number_matrix_list = np.load("ni5al5_neighbor_node_number_matrix_list.npy")


edge_feature_matrix_list = []

node_feature_matrix_list = []

neighbor_node_number_matrix_list = []

neighbor_node_index_matrix_list = []


num_graphs = 200
nodes_per_graph = 4000

# 创建一个包含所有节点的列表
all_nodes = []
for graph_id in range(num_graphs):
    graph_nodes = [graph_id] * nodes_per_graph  # 使用图的ID来表示节点所属的图
    all_nodes.extend(graph_nodes)

# 将列表转换为NumPy数组
# 定义图和节点的数量
num_graphs = 200
nodes_per_graph = 4000

# 创建一个包含所有节点的列表
all_nodes = []
for graph_id in range(num_graphs):
    graph_nodes = [graph_id] * nodes_per_graph  # 使用图的ID来表示节点所属的图
    all_nodes.extend(graph_nodes)

# 将列表转换为NumPy数组
batch = np.array(all_nodes)
batch=batch.reshape(200,4000,1)

# 打印批次的形状
#print("Batch shape:", batch.shape)

#print(batch)





for i in range(len(cu5zr5_edge_feature_matrix_list)):

    edge_feature_matrix_list.append(cu5zr5_edge_feature_matrix_list[i])

for i in range(len(ni5al5_edge_feature_matrix_list)):

    edge_feature_matrix_list.append(ni5al5_edge_feature_matrix_list[i])

#print('cu5zr5_edge_feature_matrix_list[0][0].size',cu5zr5_edge_feature_matrix_list[0][0].size)

#print('ni5al5_edge_feature_matrix_list[0][0].size',ni5al5_edge_feature_matrix_list[0][0].size)


for i in range(len(cu5zr5_node_feature_matrix_list)):

    node_feature_matrix_list.append(cu5zr5_node_feature_matrix_list[i])

for i in range(len(ni5al5_node_feature_matrix_list)):

    node_feature_matrix_list.append(ni5al5_node_feature_matrix_list[i])


for i in range(len(cu5zr5_neighbor_node_index_matrix_list)):

    neighbor_node_index_matrix_list.append(cu5zr5_neighbor_node_index_matrix_list[i])

for i in range(len(ni5al5_neighbor_node_index_matrix_list)):

    neighbor_node_index_matrix_list.append(ni5al5_neighbor_node_index_matrix_list[i])


for i in range(len(cu5zr5_neighbor_node_number_matrix_list)):

    neighbor_node_number_matrix_list.append(cu5zr5_neighbor_node_number_matrix_list[i])

for i in range(len(ni5al5_neighbor_node_number_matrix_list)):

    neighbor_node_number_matrix_list.append(ni5al5_neighbor_node_number_matrix_list[i])


atom_number = 4000

gdf = GaussianDistance(dmin=0, dmax=8, step=0.2)

edge_feature_matrix_gdf_list = []

#print(edge_feature_matrix_list[0][1].size)

#print(edge_feature_matrix_list)

#print('edge_feature_matrix_list[0][0].size',edge_feature_matrix_list[0][0].size)

for sample in range(len(edge_feature_matrix_list)):#200*4000*25*1

    edge_feature_matrix_gdf = np.zeros((atom_number, edge_feature_matrix_list[0][0].size, 41))#4000*25*41

    for number in range(atom_number):

        #print(edge_feature_matrix_list[sample][number].shape)

        gdf_edge_feature = gdf.expand(edge_feature_matrix_list[sample][number])#25*41

        #print(gdf_edge_feature.shape)

        edge_feature_matrix_gdf[number] = gdf_edge_feature.reshape(edge_feature_matrix_list[0][0].size,41)#25*41

    edge_feature_matrix_gdf_list.append(edge_feature_matrix_gdf)#200*4000*25*41

#print(edge_feature_matrix_gdf_list[0][0])

ln_Label = np.zeros((200,1))

for sample in range(200):

    if sample < 100:#Cu_Zr:(0,1)

        ln_Label[sample][0] = 1

    elif sample >= 100:#Al_Ni:(1,0)

        ln_Label[sample][0] = 0


edge_feature_matrix = np.zeros((len(edge_feature_matrix_list),atom_number,edge_feature_matrix_list[0][0].size,41))#200*4000*25*41

neighbor_mask_matrix = np.zeros((len(edge_feature_matrix_list),atom_number,edge_feature_matrix_list[0][0].size))#200*4000*25

neighbor_index_matrix = np.zeros((len(edge_feature_matrix_list),atom_number,edge_feature_matrix_list[0][0].size))#200*4000*25

node_feature_matrix = np.zeros((len(edge_feature_matrix_list),atom_number,2))#200*4000*2

for i in range(len(edge_feature_matrix_list)):#200

    edge_feature_matrix[i] = edge_feature_matrix_gdf_list[i]#200*4000*25*41

    neighbor_mask_matrix[i] = neighbor_node_number_matrix_list[i]#200*4000*25

    neighbor_index_matrix[i] = neighbor_node_index_matrix_list[i]#200*4000*25

    node_feature_matrix[i] = node_feature_matrix_list[i]#200*4000*2


#sample_edge_feature_matrix = padding_zeros_matrix(matrix_list = edge_feature_matrix_gdf_list)

#print(sample_edge_feature_matrix[0][0][0])

#sample_node_feature_matrix = padding_zeros_matrix(matrix_list = node_feature_matrix_list)

#sample_neighbor_node_number_matrix = padding_zeros_matrix(matrix_list = neighbor_node_number_matrix_list)

#sample_neighbor_node_index_matrix = padding_zeros_matrix(matrix_list = neighbor_node_index_matrix_list)

#sample_contre_node_index_matrix = padding_zeros_matrix(matrix_list = contre_node_index_matrix_list)

#num_edge_number_matrix = edge_number_matrix(matrix_list = edge_feature_matrix_list)

#print('num_edge_number_matrix',num_edge_number_matrix)

#Cu50_GFA_label = GFA_label(sample_number=set_num_sample,thickness=1.13)

sample_edge_feature_matrix_tensor = torch.from_numpy(edge_feature_matrix).float()

#print(sample_edge_feature_matrix_tensor[0][0])

sample_node_feature_matrix_tensor = torch.from_numpy(node_feature_matrix).float()

sample_neighbor_mask_matrix_tensor = torch.from_numpy(neighbor_mask_matrix).float()

sample_neighbor_node_index_matrix_tensor = torch.from_numpy(neighbor_index_matrix).float()

#sample_contre_node_index_matrix_tensor = torch.from_numpy(sample_contre_node_index_matrix).float()

sample_label_tensor = torch.from_numpy(ln_Label).float()

batch_tensor = torch.from_numpy(batch).float()

#print('batch_tensor',batch_tensor)

#edge_number_matrix_trnsor = torch.from_numpy(con_edge_number_matrix).float()

#GFA_label_tensor = torch.from_numpy(con_GFA_label).float()


m_node=deepcopy(sample_node_feature_matrix_tensor[0])

m_edge=deepcopy(sample_edge_feature_matrix_tensor[0])

m_mask=deepcopy(sample_neighbor_mask_matrix_tensor[0])

m_index=deepcopy(sample_neighbor_node_index_matrix_tensor[0])

n_node=deepcopy(sample_node_feature_matrix_tensor[100])

n_edge=deepcopy(sample_edge_feature_matrix_tensor[100])

n_mask=deepcopy(sample_neighbor_mask_matrix_tensor[100])

n_index=deepcopy(sample_neighbor_node_index_matrix_tensor[100])



#尝试分出测试集
train_lengths = 150

validation_lengths = 50

#交叉验证

train_edge_feature_matrix_tensor, validation_edge_feature_matrix_tensor = random_split(dataset=sample_edge_feature_matrix_tensor,
                                                                   lengths=[train_lengths,validation_lengths],
                                                                   generator=torch.Generator().manual_seed(5))

train_node_feature_matrix_tensor, validation_node_feature_matrix_tensor = random_split(dataset=sample_node_feature_matrix_tensor,
                                                                   lengths=[train_lengths,validation_lengths],
                                                                   generator=torch.Generator().manual_seed(5))

train_neighbor_node_mask_matrix_tensor, validation_neighbor_node_mask_matrix_tensor = random_split(dataset=sample_neighbor_mask_matrix_tensor,
                                                                   lengths=[train_lengths,validation_lengths],
                                                                   generator=torch.Generator().manual_seed(5))

train_neighbor_node_index_matrix_tensor, validation_neighbor_node_index_matrix_tensor = random_split(dataset=sample_neighbor_node_index_matrix_tensor,
                                                                    lengths=[train_lengths,validation_lengths],
                                                                    generator=torch.Generator().manual_seed(5))

#train_contre_node_index_matrix_tensor, validation_contre_node_index_matrix_tensor = random_split(dataset=sample_contre_node_index_matrix_tensor,
#                                                                    lengths=[train_lengths,validation_lengths],
#                                                                    generator=torch.Generator().manual_seed(7))


train_sample_label_tensor , validation_sample_label_tensor = random_split(dataset=sample_label_tensor,
                                                                    lengths=[train_lengths,validation_lengths],
                                                                    generator=torch.Generator().manual_seed(5))

train_batch_tensor, validation_batch_tensor = random_split(dataset=batch_tensor,
                                                                    lengths=[train_lengths,validation_lengths],
                                                                    generator=torch.Generator().manual_seed(5))

print('########################train_batch_tensor',train_batch_tensor)

#train_GFA_label_tensor, validation_GFA_label_tensor = random_split(dataset=GFA_label_tensor,
#                                                    lengths=[train_lengths,validation_lengths],
#                                                    generator=torch.Generator().manual_seed(7))

                                                                

#print(Cu50_neighbor_number_matrix_tensor)

''''normalizer = Normalizer(train_edge_feature_matrix_tensor)

train_edge_feature_matrix_tensor = normalizer.norm(train_edge_feature_matrix_tensor)
validation_edge_feature_matrix_tensor = normalizer.norm(validation_edge_feature_matrix_tensor)'''




train_dataset = Data.TensorDataset(train_edge_feature_matrix_tensor[:] ,train_node_feature_matrix_tensor[:], 
                                  train_neighbor_node_mask_matrix_tensor[:],train_neighbor_node_index_matrix_tensor[:],
                                  train_sample_label_tensor[:])

train_loader = Data.DataLoader(dataset=train_dataset,batch_size=1,shuffle=True,num_workers=0)  #数据采样

validation_dataset = Data.TensorDataset(validation_edge_feature_matrix_tensor[:] ,validation_node_feature_matrix_tensor[:], 
                                  validation_neighbor_node_mask_matrix_tensor[:],validation_neighbor_node_index_matrix_tensor[:],
                                  validation_sample_label_tensor[:])
validation_loader = Data.DataLoader(dataset=validation_dataset,batch_size=1,shuffle=True,num_workers=0)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def test(model,loader):

    true_out = []

    Pre_out = []

    Pre_loss = []

    test_losses = 0

    for data in loader:#20

        edge_features, node_features, neighbor_masks, neighbor_indexs, label = data
        edge_features, node_features, neighbor_masks, neighbor_indexs, label = edge_features.to(device), node_features.to(device), neighbor_masks.to(device), neighbor_indexs.to(device), label.to(device)


        #print('##########################data[4]',data[4])

        #centre_node_index_matrix_tensor = data[4].to(device)

        out,a = model(node_features[0],edge_features[0],neighbor_indexs[0],neighbor_masks[0])

        Pre_out.append(out)
        
        #print('test_pred',pred)
        
        y = label[0].to(device)#1*2

        true_out.append(y)
        
        #print('test_true',y)
        
        loss_function = torch.nn.NLLLoss()#BCEWithLogitsLoss
        
        loss = loss_function(out, y.to(torch.long))

        test_losses = loss + test_losses

        Pre_loss.append(loss)
        #print(pred)
    return Pre_out,Pre_loss,true_out, test_losses


#GNN_layer_1 = GNNLayer(node_embedding_len=2,edge_embedding_len=1,attention_len=1)

#GNN_layer_2 = GNNLayer(node_embedding_len=2,edge_embedding_len=1,attention_len=1)

#print(model)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = GNNModel(node_feature_len=2,edge_embedding_len=41,init_node_embedding_units=[2])


model = model.to(device) 


optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)




Epoch = []

Train_loss = []

Test_loss = []

RESUME = False

EPOCH = 4

start_epoch = -1

if RESUME:
    path_checkpoint = "./models/checkpoint/ckpt_best_1735.pth"  # 断点路径
    checkpoint = torch.load(path_checkpoint)  # 加载断点

    model.load_state_dict(checkpoint['net'])  # 加载模型可学习参数

    optimizer.load_state_dict(checkpoint['optimizer'])  # 加载优化器参数
    start_epoch = checkpoint['epoch']  # 设置开始的epoch
    

for epoch in range(start_epoch+1, EPOCH):

    print('epoch',epoch)

    losses = 0

    for step, data in enumerate(train_loader):

        #print('step',step)


        #data = torch.stack(data, dim=1)
        edge_features, node_features, neighbor_masks, neighbor_indexs, label = data
        edge_features, node_features, neighbor_masks, neighbor_indexs, label = edge_features.to(device), node_features.to(device), neighbor_masks.to(device), neighbor_indexs.to(device), label.to(device)

        #4000*2

        #4000*25*41

        #4000*25

        #4000*25

        #4000*1

        #print('##########################data[4]',data[4])

        #centre_node_index_matrix_tensor = data[4].to(device)

        out,a = model(node_features[0],edge_features[0],neighbor_indexs[0],neighbor_masks[0])

        #print('out',out)

        #print('y',y)

        optimizer.zero_grad()

        loss_function = torch.nn.NLLLoss()


        loss = loss_function(out, label[0].to(torch.long))

        

        loss.backward()
        
        optimizer.step()

        losses = losses + loss

        #print('loss',loss)

    #val_loss = test(model,validation_loader)

    #print('val_loss', val_loss)

    print('losses',losses.item())

    #pccs = np.corrcoef(out.cpu().detach().numpy().reshape(1,200), y.cpu().cpu().detach().numpy().reshape(1,200))

    #print('pccs',pccs)

    Epoch.append(epoch)

    Train_loss.append(losses.item())

    out,loss,true_out,test_losses = test(model,validation_loader)
    
    target_tensor = torch.tensor(true_out).unsqueeze(1) 
    
    prediction_tensor = torch.stack([torch.squeeze(out[i]) for i in range(len(out))])
    
    accuracy, precision, recall, fscore, auc_score = classification_eval(target_tensor, prediction_tensor)

   #print(f"Accuracy: {accuracy}")
    #print(f"Precision: {precision}")
    #print(f"Recall: {recall}")
    #print(f"F1 Score: {fscore}")
    #print(f"AUC Score: {auc_score}")

#print(out,loss,true_out)
    print('test_losses',test_losses.item())
    #print(out[2].cpu().detach().numpy())
    #print(loss[2].cpu().detach().numpy())
    #print(true_out[2].cpu().detach().numpy())

    #test_pccs = np.corrcoef(out[0].cpu().detach().numpy().reshape(1,200), true_out[0].cpu().detach().numpy()[0].reshape(1,200))

    #print('test_pcc',test_pccs)

    Test_loss.append(test_losses.item())

    if epoch %10 == 0:


        #print('epoch', epoch)

        #print('loss',loss)
        #print('out',out)
        #print('y',y)

        print('save_epoch:',epoch)

        checkpont = {
            "net": model.state_dict(),
            'optimizer':optimizer.state_dict(),
            "epoch":epoch
        }
        #if not os.path.isdir("./models/checkpoint"):
            #os.mkdir('./models/checkpoint')
        #torch.save(checkpont,'./models/checkpoint/ckpt_best_%s.pth'%(str(epoch)))



    if epoch == 3:

        torch.save(model.state_dict(),'test.pth')

#import seaborn as sns  # 导入Seaborn库


alpha_value = 0.5 
# 随机生成[0, 4000]范围内的100个不重复的整数
random_numbers = random.sample(range(0, 4001), 100)

print(random_numbers)


plt.clf()
plt.cla()

# 初始化图
G = nx.Graph()

# 假设model, n_node, n_edge, n_index, n_mask已经定义并准备好
j2, attention_weight = model(m_node.to(device), m_edge.to(device), m_index.to(device), m_mask.to(device))

# 设置一个权重的阈值，这里我们选择仅显示最重要的10%的边
all_weights = attention_weight.view(-1).cpu().detach().numpy()  # 转换权重为1D numpy数组
threshold = np.percentile(all_weights, 95)  # 计算90%分位数作为阈值

# 初始化图，这次我们只添加权重大于等于阈值的边
for i in range(4000):  # 假设N和n_index.shape[1]已经定义
    for j in range(m_index.shape[1]):
        node1 = i
        node2 = m_index[i, j].item()
        weight = attention_weight[i, j, 0].item()
        
        if weight >= threshold:  # 仅当权重大于等于阈值时，添加边
            G.add_node(node1)  # 添加节点，networkx会自动忽略重复的节点
            G.add_node(node2)
            G.add_edge(node1, node2, weight=weight)  # 添加带权重的边

# 创建一个更大的图形来绘制
plt.figure(figsize=(100, 100))

# 计算节点位置
pos= nx.spring_layout(G, k=0.3, scale=2.0, iterations=10)

# 绘制节点
nx.draw_networkx_nodes(G, pos, node_color='red', node_size=100)

# 绘制过滤后的边，调整边的粗细基于权重
edges = G.edges(data=True)
for (node1, node2, data) in edges:
    nx.draw_networkx_edges(G, pos, edgelist=[(node1, node2)], width=data['weight']*40)

# 移除坐标轴
plt.axis('off')

# 保存图形
plt.savefig('11.png')

plt.close()




plt.clf()
plt.cla()

# 初始化图
G = nx.Graph()

# 假设model, n_node, n_edge, n_index, n_mask已经定义并准备好
j2, attention_weight = model(n_node.to(device), n_edge.to(device), n_index.to(device), n_mask.to(device))

# 设置一个权重的阈值，这里我们选择仅显示最重要的10%的边
all_weights = attention_weight.view(-1).cpu().detach().numpy()  # 转换权重为1D numpy数组
threshold = np.percentile(all_weights, 95)  # 计算90%分位数作为阈值

# 初始化图，这次我们只添加权重大于等于阈值的边
for i in range(4000):  # 假设N和n_index.shape[1]已经定义
    for j in range(n_index.shape[1]):
        node1 = i
        node2 = n_index[i, j].item()
        weight = attention_weight[i, j, 0].item()
        
        if weight >= threshold:  # 仅当权重大于等于阈值时，添加边
            G.add_node(node1)  # 添加节点，networkx会自动忽略重复的节点
            G.add_node(node2)
            G.add_edge(node1, node2, weight=weight)  # 添加带权重的边

# 创建一个更大的图形来绘制
plt.figure(figsize=(100, 100))

# 计算节点位置
pos= nx.spring_layout(G, k=0.3, scale=2.0, iterations=10)

# 绘制节点
nx.draw_networkx_nodes(G, pos, node_color='yellow', node_size=100)

# 绘制过滤后的边，调整边的粗细基于权重
edges = G.edges(data=True)
for (node1, node2, data) in edges:
    nx.draw_networkx_edges(G, pos, edgelist=[(node1, node2)], width=data['weight']*40)

# 移除坐标轴
plt.axis('off')

# 保存图形
plt.savefig('12.png')

plt.close()




plt.figure()
# 假设Train_loss是需要梯度的张量
plt.plot(Epoch, Train_loss, c='r', label='Train Loss')
# 假设Train_loss是需要梯度的张量
plt.plot(Epoch, Test_loss, c='b', label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Train vs Test Loss')
plt.legend()
plt.savefig('test.png')
print("##################################")






/home/ma-user/anaconda3/envs/PyTorch-1.8/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


###################################################Save_data_success#############################################################
########################train_batch_tensor <torch.utils.data.dataset.Subset object at 0x7f4d37c540d0>
epoch 0
losses 73.29076385498047
test_losses 4.808205604553223
save_epoch: 0
epoch 1
losses 3.722734212875366
test_losses 0.31362947821617126
epoch 2
losses 0.5975677371025085
test_losses 0.1292521357536316
epoch 3
losses 0.285690575838089
test_losses 0.07177551835775375
[1353, 2413, 952, 1259, 1128, 142, 1497, 3060, 1975, 3097, 3765, 2096, 2524, 1470, 1022, 1153, 3703, 1800, 1298, 1309, 1745, 764, 2454, 2716, 3587, 2927, 1686, 2585, 345, 2758, 3470, 3377, 18, 138, 927, 3943, 3526, 1491, 1276, 2479, 3173, 2494, 1993, 323, 2476, 1413, 2680, 2847, 140, 67, 3222, 2371, 2117, 476, 3977, 240, 3213, 3960, 2459, 3428, 118, 990, 1575, 2464, 495, 2939, 2639, 2149, 3853, 1566, 3695, 1156, 236, 2289, 2949, 3473, 3516, 3058, 2912, 3498, 1704, 2367, 3739, 722, 3826, 2411